In [13]:
# Librerias necesarias
import os
import json
import random

In [14]:
# Archivos necesarios, cargo los diccionarios de la carpeta "resultados incertidumbre", en orden de uso
with open("../1. codigo analisis/resultados incertidumbre/llegadas.json", "r") as file:
    llegadas = json.load(file)
with open("../1. codigo analisis/resultados incertidumbre/matrices.json", "r") as file:
    matrices = json.load(file)
with open("../1. codigo analisis/resultados incertidumbre/los.json", "r") as file:
    los = json.load(file)
with open("../1. codigo analisis/resultados incertidumbre/los_OR.json", "r") as file:
    los_OR = json.load(file)

In [15]:
# Funciones utilizadas
def test_uniforme(uniforme, final_kde_pmf, inicio):
    inicio = inicio
    cumulative_probability = 0
    for index, probability in enumerate(final_kde_pmf):
        cumulative_probability += probability
        if uniforme < cumulative_probability:
            return index + inicio
        elif uniforme == 1:
            return len(final_kde_pmf) - (1 - inicio) # En caso de que el uniforme sea 1, se devuelve el último índice

def dict_matriz_a_pmf(matriz_dict):
    pmf = [[], [], []]
    for unidad_inicio in range(1,4):
        for unidad_termino in range(1,4):
            pmf[unidad_inicio-1].append(matriz_dict[str(unidad_inicio)][str(unidad_termino)])      
    return pmf

# Le entrego la matriz con las transiciones posibles, la unidad donde parte y devuelve un camino
def camino_matriz(matriz, unidad_actual):
    camino = []
    camino.append(unidad_actual)
    termino = False
    while not termino:
        uniforme = random.uniform(0, 1)
        unidad_siguiente = test_uniforme(uniforme, matriz[unidad_actual-1], 1)
        if unidad_siguiente == unidad_actual: # Solo ocurre de SDU_WARD a SDU_WARD
            termino = True
        else:
            camino.append(unidad_siguiente)
            unidad_actual = unidad_siguiente
    return camino

# Le entrego el camino que siguio cierto paciente y me entrega el tiempo en cada unidad
def calculo_los(los, los_OR, hospital, grd, camino):
    tiempo_en_camino = []
    pmf_OR = los_OR[str(hospital)][str(grd)]["final_kde_pmf"].copy()
    pmf_ICU = los[str(hospital)]["ICU"][str(grd)]["final_kde_pmf"].copy()
    pmf_SDU = los[str(hospital)]["SDU_WARD"][str(grd)]["final_kde_pmf"].copy()
    pmfs = {
        1: pmf_OR,
        2: pmf_ICU,
        3: pmf_SDU
    }
    for unidad in camino:
        uniforme = random.uniform(0, 1)
        tiempo_en_camino.append(test_uniforme(uniforme, pmfs[unidad], 1))
    
    return tiempo_en_camino

In [16]:
"""Definir semilla para random, muy importante para reproducibilidad de resultados, son 48 sub listas
de llegadas, ya que del grd 5 al 8 tienen 3 requerimientos, osea 12 sub listas. Del 1 al 4 tienen
3 requerimientos pero ademas llegan a 3 hospitales diferentes por lo que son 36 sub listas. Esto da
un total de 48 sub listas. Cada sub lista se crea secuancialmente, se llena una y se pasa a la siguiente. """
random.seed(42)
cantidad_de_ciclos = 5000 # Cantidad de ciclos a simular, se puede cambiar

"""En primer se simulan todas las llegadas, ya sea a WL o a los distintos ED
Se separan por hospital (o WL), requerimiento inicial y grd"""
llegadas_simuladas = {}
for hospital in range(0,4): # El cero hace referencia a la WL
    llegadas_simuladas[hospital] = {}
    for requerimiento in range(1, 4):
        llegadas_simuladas[hospital][requerimiento] = {}
        for grd in range(1, 9):
            llegadas_simuladas[hospital][requerimiento][grd] = []
            if llegadas[str(hospital)][str(requerimiento)][str(grd)] != {}: # No considero los vacios de llegadas.json
                for ciclo in range(1, cantidad_de_ciclos + 1):
                    uniforme = random.uniform(0, 1) # Genero un numero aleatorio entre 0 y 1
                    final_kde_pmf = llegadas[str(hospital)][str(requerimiento)][str(grd)]["final_kde_pmf"] # Accedo a la lista de probabilidades
                    numero_llegadas = test_uniforme(uniforme, final_kde_pmf, inicio = 0) # Parte desde las cero llegadas
                    for persona in range(numero_llegadas):    
                        llegadas_simuladas[hospital][requerimiento][grd].append(ciclo) # Agrego el ciclo al que llega la persona

"""Siguiendo la misma logica anterior, por cada paciente se generan 3 posibles caminos,
uno por cada hospital, ya que no se sabe en que hospital terminaran siendo
atendidos (si es que no son enviados al PS)"""
transiciones_simuladas = {}
for hospital in range(0,4): # 0 son llegadas a WL
    transiciones_simuladas[hospital] = {}
    for requerimiento in range(1,4):
        transiciones_simuladas[hospital][requerimiento] = {}
        for grd in range(1,9):
            transiciones_simuladas[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(llegadas_simuladas[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                for paciente in range(cantidad_pacientes):
                    matrices_posibles = {
                        1: camino_matriz(dict_matriz_a_pmf(matrices["1"][str(grd)]), requerimiento),
                        2: camino_matriz(dict_matriz_a_pmf(matrices["2"][str(grd)]), requerimiento),
                        3: camino_matriz(dict_matriz_a_pmf(matrices["3"][str(grd)]), requerimiento)
                    }
                    transiciones_simuladas[hospital][requerimiento][grd].append(matrices_posibles)

"""Finalmente teniendo cada uno de los caminos, se calcula el tiempo que se estara en cada una
de las unidades, de esta manera cada camino de unidades tiene asociado un camino de tiempo"""
los_simuladas = {}
for hospital in range(0,4): # 0 son llegadas a WL
    los_simuladas[hospital] = {}
    for requerimiento in range(1,4):
        los_simuladas[hospital][requerimiento] = {}
        for grd in range(1,9):
            los_simuladas[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(llegadas_simuladas[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                for camino_paciente in transiciones_simuladas[hospital][requerimiento][grd]:
                    los_posibles = {
                        1: calculo_los(los, los_OR, 1, grd, camino_paciente[1]),
                        2: calculo_los(los, los_OR, 2, grd, camino_paciente[2]),
                        3: calculo_los(los, los_OR, 3, grd, camino_paciente[3])
                    }
                    los_simuladas[hospital][requerimiento][grd].append(los_posibles)

"""Uno todo por cada uno de los pacientes, de esa manera instanciar a los paciente mas adelante sera
facil ya que cada uno cuenta con toda la información que se necesita"""
incertidumbre_pacientes = {}
for hospital in range(0,4): # 0 son llegadas a WL
    incertidumbre_pacientes[hospital] = {}
    for requerimiento in range(1,4):
        incertidumbre_pacientes[hospital][requerimiento] = {}
        for grd in range(1,9):
            incertidumbre_pacientes[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(llegadas_simuladas[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                l_s = llegadas_simuladas[hospital][requerimiento][grd]
                t_s = transiciones_simuladas[hospital][requerimiento][grd]
                lo_s = los_simuladas[hospital][requerimiento][grd]
                for ti, transicion, espera in zip(l_s, t_s, lo_s):
                    unido = {"TI": ti,
                    "camino": transicion,
                    "espera": espera}
                    incertidumbre_pacientes[hospital][requerimiento][grd].append(unido)

# Asi se ven los datos para cada paciente
# incertidumbre_pacientes

In [ ]:
# Guardar el diccionario en un archivo JSON
def save_dict_as_json(data_dict, filename, folder):
    os.makedirs(folder, exist_ok=True)  # Create folder if it doesn't exist
    path = os.path.join(folder, filename)
    with open(path, 'w') as f:
        json.dump(data_dict, f, indent=4)
    print(f"Dictionary saved to: {path}")

# Save the results to a JSON file
save_dict_as_json(incertidumbre_pacientes, filename="incertidumbre_simulada.json", folder="resultados incertidumbre")

Dictionary saved to: resultados incertidumbre/incertidumbre_simulada_1.json


In [18]:
"""Se puede ver que por cada lista con las transiciones, que tienen como llave el hospital,
hay otra lista que hace referencia al los en esa unidad, si se descomenta el codigo de abajo se
puede ver que por cada paciente se generaron la misma cantidad de transiciones y tiempos de espera. 
La relación es que tienen el mismo indice en cada una de sus listas, por ejemplo:

llegadas_simuladas[0][1][5][0]
El paciente que llego en el ciclo: 
1

transiciones_simuladas[0][1][5][0]
Tendra las transiciones: {
1: [1, 3], 
2: [1, 2, 3], 
3: [1, 2, 3]
}

los_simuladas[0][1][5][0]
Cada una con tiempos de espera:{
1: [1, 7], 
2: [1, 25, 15], 
3: [1, 2, 11]
}

De esta manera todo lo relacionado a incertidumbre ya se sabe antes de partir la simulación
"""

print(llegadas_simuladas[0][2][5])
print(transiciones_simuladas[0][2][5])
print(los_simuladas[0][2][5])

# Corroboro haber generado las matrices y los correspondientes para cada paciente (descomentar para probar)
# for hospital in range(0,4): # 0 son llegadas a WL
#     for requerimiento in range(1,4):
#         for grd in range(1,9):
#             n_llegadas = len(llegadas_simuladas[hospital][requerimiento][grd])
#             n_transiciones = len(transiciones_simuladas[hospital][requerimiento][grd])
#             n_los = len(los_simuladas[hospital][requerimiento][grd])
#             # print(f"personas: {n_llegadas}, trancisiones: {n_transiciones}, dif: {dif}")
#             print(f"n: {n_llegadas}, t: {n_transiciones}, los: {n_los}")

[1, 1, 1, 1, 2, 2, 3, 4, 5, 7, 7, 9, 9, 9, 10, 10, 12, 13, 14, 15, 15, 15, 16, 16, 17, 19, 20, 20, 20, 20, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 29, 29, 29, 32, 32, 32, 34, 34, 34, 34, 35, 36, 37, 37, 37, 38, 39, 39, 41, 43, 44, 44, 45, 46, 46, 46, 47, 49, 50, 51, 51, 51, 51, 51, 52, 52, 53, 53, 55, 55, 55, 55, 55, 56, 57, 57, 58, 59, 59, 59, 61, 61, 62, 62, 62, 63, 63, 64, 64, 64, 64, 65, 65, 65, 66, 66, 66, 66, 67, 67, 67, 69, 69, 69, 69, 70, 71, 71, 71, 72, 72, 72, 72, 73, 73, 74, 74, 74, 74, 75, 76, 77, 78, 78, 78, 79, 80, 80, 80, 80, 81, 82, 84, 84, 84, 86, 87, 88, 88, 88, 88, 88, 89, 89, 89, 89, 91, 92, 92, 93, 93, 93, 94, 94, 94, 95, 97, 97, 97, 97, 98, 98, 99, 99, 100, 100, 100, 100, 100, 100, 101, 102, 102, 103, 104, 104, 106, 106, 107, 107, 107, 107, 107, 107, 108, 108, 109, 109, 111, 111, 111, 111, 113, 114, 116, 116, 116, 116, 116, 117, 117, 118, 120, 121, 122, 123, 124, 125, 125, 125, 126, 126, 127, 128, 128, 128, 128, 128, 129, 129, 130, 131,